In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import os 
import seaborn as sns
from sklearn.model_selection import train_test_split
import pickle
import tensorflow as tf
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler,  RobustScaler
import pickle
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from keras import regularizers, layers, optimizers, initializers
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error


from tensorflow.keras.applications import EfficientNetV2M, ConvNeXtBase, ConvNeXtLarge, ConvNeXtXLarge
import numpy as np
import gc
import tensorflow_addons as tfa
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Input, Concatenate, Dropout, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(f'Current GPU allocator: {os.getenv("TF_GPU_ALLOCATOR")}')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            print(f'Setting memory growth for {gpu}')
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
    

2024-05-30 17:05:48.897789: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 17:05:51.722670: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/tobias/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Current GPU allocator: cuda_malloc_async
Setting memory growth for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-05-30 17:05:53.481694: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-30 17:05:53.682260: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-30 17:05:53.682355: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
study_name = '527_convnextl_0'

In [3]:

mean_columns = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']

In [4]:
pd.set_option('display.max_rows', None)

# Aseta näyttämään rajoittamaton määrä sarakkeita
pd.set_option('display.max_columns', None)

In [5]:

pickle_file_path = './data/test_df_kolmas.pickle'

with open(pickle_file_path, 'rb') as f:
    test_df = pickle.load(f)

pickle_file_path = './data/train_df_kolmas.pickle'

with open(pickle_file_path, 'rb') as f:
    train_df = pickle.load(f)

In [ ]:
test_df.head()

In [ ]:
train_df.head()

In [6]:
train_original = train_df.copy()
train_plot = train_df.copy()
sample_df = train_df.copy()

train_df = sample_df[sample_df.fold != 0]
valid_df = sample_df[sample_df.fold == 0]

print(f"# Num Train: {len(train_df)} | Num Valid: {len(valid_df)}")


# Num Train: 42308 | Num Valid: 10577


In [9]:
import glob

directory_path = './NN_search'
pattern = f"{directory_path}/{study_name}*.h5"

files = glob.glob(pattern)

max_r2_score = float('-inf')
best_model = None

# Käy läpi jokainen tiedosto ja etsi suurin r2_score_inv
for file in files:
    value = float(file.split('best_val')[1].split('_')[1])
    if value > max_r2_score:
        max_r2_score = value
        best_model = file


# Tulosta suurin löydetty r2_score_inv ja vastaava tiedosto
print(f"Best R2-score: {max_r2_score:.5f}")
if best_model:
    print(f"Best model: {best_model}")
else:
    print("No best model found")

best_log_transforms_name =  f'./NN_search/{study_name}_{max_r2_score:.5f}_best_log_transforms.pickle'
best_scalers_name = f'./NN_search/{study_name}_{max_r2_score:.5f}_best_scalers.pickle'

print(f'Opening log transforms from {best_log_transforms_name}')
with open(best_log_transforms_name, 'rb') as f:
    log_transforms = pickle.load(f)

print(f'Opening scalers from {best_scalers_name}')
with open(best_scalers_name, 'rb') as f:
    scaler_transforms = pickle.load(f)


def r2_score_tf(y_true, y_pred):

    try: 
        ss_res = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
        ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)
        r2 = 1 - ss_res/(ss_tot + tf.keras.backend.epsilon())
        r2 = tf.where(tf.math.is_nan(r2), tf.zeros_like(r2), r2) 
        return tf.reduce_mean(tf.maximum(r2, 0.0))
    except Exception as e:
        # print(f'Error in r2_score_tf: {e}')
        return float('-inf')
    
custom_objects = {"r2_score_tf": r2_score_tf}

        

Best R2-score: 0.34319
Best model: ./NN_search/527_convnextl_0_best_val_0.34319_model.h5
Opening log transforms from ./NN_search/527_convnextl_0_0.34319_best_log_transforms.pickle
Opening scalers from ./NN_search/527_convnextl_0_0.34319_best_scalers.pickle


In [ ]:
print(f'Scaler are: {scaler_transforms}')
print(f'Log transforms are: {log_transforms}')

In [7]:
def random_erasing(img, probability=0.5, sl=0.02, sh=0.4, r1=0.3, method = 'black'):

    if tf.random.uniform([]) > probability:
        return img

    img_width = img.shape[1]
    img_height = img.shape[0]
    img_channels = img.shape[2]

    area = img_height * img_width

    target_area = tf.random.uniform([], minval=sl, maxval=sh) * area
    aspect_ratio = tf.random.uniform([], minval=r1, maxval=1/r1)
    h = tf.cast(tf.math.round(tf.math.sqrt(target_area * aspect_ratio)), tf.int32)
    w = tf.cast(tf.math.round(tf.math.sqrt(target_area / aspect_ratio)), tf.int32)

    while tf.constant(True, dtype=tf.bool):
        if h > img_height or w > img_width:
            target_area = tf.random.uniform([], minval=sl, maxval=sh) * area
            aspect_ratio = tf.random.uniform([], minval=r1, maxval=1/r1)
            h = tf.cast(tf.math.round(tf.math.sqrt(target_area * aspect_ratio)), tf.int32)
            w = tf.cast(tf.math.round(tf.math.sqrt(target_area / aspect_ratio)), tf.int32)
        else:
            break

    x1 = tf.cond(img_height == h, lambda: 0, lambda: tf.random.uniform([], minval=0, maxval=img_height - h, dtype=tf.int32))
    y1 = tf.cond(img_width == w, lambda: 0, lambda: tf.random.uniform([], minval=0, maxval=img_width - w, dtype=tf.int32))

    part1 = tf.slice(img, [0, 0, 0], [x1, img_width, img_channels])
    part2 = tf.slice(img, [x1, 0, 0], [h, y1, img_channels])

    # Initialize part3 with default values
    part3 = tf.zeros((h, w, img_channels), dtype=tf.float32)
    
    if method == 'black':
        part3 = tf.zeros((h, w, img_channels), dtype=tf.float32)
    elif method == 'white':
        part3 = tf.ones((h, w, img_channels), dtype=tf.float32)
    elif method == 'random':
        part3 = tf.random.uniform((h, w, img_channels), dtype=tf.float32)

    part4 = tf.slice(img, [x1, y1 + w, 0], [h, img_width - y1 - w, img_channels])
    part5 = tf.slice(img, [x1 + h, 0, 0], [img_height - x1 - h, img_width, img_channels])

    middle_row = tf.concat([part2, part3, part4], axis=1)
    img = tf.concat([part1, middle_row, part5], axis=0)

    return img

def augment_image(img):

    img = img / 255.0

    img = random_erasing(img, probability=0.5, sl=0.05, sh=0.3, r1=0.5)

    crop_size = tf.random.uniform(shape=[], minval=420, maxval=480, dtype=tf.int32)
    img = tf.image.random_crop(img, size=[crop_size, crop_size, 3])
    img = tf.image.resize(img, [480, 480]) 
    
    img = tf.image.random_flip_left_right(img)
    
    img = tf.image.rot90(img, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

    img = tf.image.random_brightness(img, max_delta=0.2)
    img = tf.image.random_hue(img, max_delta=0.2)
    img = tf.image.random_saturation(img, lower=0.8, upper=1.2)
    img = tf.image.random_contrast(img, lower=0.8, upper=1.2)

    img = tf.image.random_jpeg_quality(img, min_jpeg_quality=85, max_jpeg_quality=100)

    img = tfa.image.shear_x(img, level=tf.random.uniform([], minval=-0.2, maxval=0.2), replace=0)
    img = tfa.image.shear_y(img, level=tf.random.uniform([], minval=-0.2, maxval=0.2), replace=0)
    
    angle = tf.random.uniform([], minval=-np.pi/4, maxval=np.pi/4, dtype=tf.float32)
    img = tfa.image.rotate(img, angle)

    img = tf.image.resize(img, [480, 480]) 
    img = img * 255.0

    return img


def process_image(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (480, 480))
    img = augment_image(img)  
    # img = tf.cast(img, tf.uint8)    

    # tf.print("Final min and max in process_images:", tf.reduce_min(img), tf.reduce_max(img))
    # tf.print("Image type: ", img.dtype)

    return img

def process_image_valid(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (480, 480))
    # tf.print("Final min and max in process_image_valid:", tf.reduce_min(img), tf.reduce_max(img))
    # img = tf.cast(img, tf.uint8)
    return img

# Define your dataset processing function
def process_path_train(file_path, targets):
    img = process_image(file_path)
    return img, targets


def process_path_valid(file_path, targets):
    img = process_image_valid(file_path)
    return img, targets

def process_path_test(file_path, dummy):
    img = process_image_valid(file_path)    
    return img, dummy


In [10]:
y_train = train_df[mean_columns]
y_valid = valid_df[mean_columns]

y_train_transformed = y_train.copy()
y_valid_transformed = y_valid.copy()


transformation_options = {
    'none': lambda x: x,
    'log': lambda x, base: np.log(x) / np.log(base),
    'power': lambda x, exp: np.power(x, exp)
}

# Simuloi datan muunnos ja arvioi mallia
for target in mean_columns:
    method, param = log_transforms[target]
    if method != 'none':
        print(f'Transforming {target} with {method} and param {param}')
        y_train_transformed[target] = transformation_options[method](y_train[target], param)
        y_valid_transformed[target] = transformation_options[method](y_valid[target], param)
        
    else:
        print(f'No transformation for {target}')
        y_train_transformed[target] = y_train[target]
        y_valid_transformed[target] = y_valid[target]

for target, scaler in scaler_transforms.items():
    if scaler is not None:
        print(f'Scaling {target} with {scaler}')
        y_train_transformed[target] = scaler.fit_transform(y_train_transformed[target].values.reshape(-1, 1)).flatten()
        y_valid_transformed[target] = scaler.transform(y_valid_transformed[target].values.reshape(-1, 1)).flatten()

        



No transformation for X4_mean
No transformation for X11_mean
No transformation for X18_mean
Transforming X50_mean with log and param 4
No transformation for X26_mean
No transformation for X3112_mean
Scaling X3112_mean with StandardScaler()


In [11]:
BUFFER_SIZE = 55000 
EPOCHS = 15 
BATCH_SIZE = 6

train_images_path = train_df['image_path'].values
valid_images_path = valid_df['image_path'].values

train_dataset = tf.data.Dataset.from_tensor_slices((train_images_path, y_train_transformed))
train_dataset = train_dataset.shuffle(BUFFER_SIZE)

valid_dataset = tf.data.Dataset.from_tensor_slices((valid_images_path, y_valid_transformed))

train_dataset = train_dataset.map(process_path_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.map(process_path_valid, num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

len_dataset = len(train_dataset) 
len_all_train = len(train_dataset) * EPOCHS

print(f'Len dataset: {len_dataset}')
print(f'Len all train: {len_all_train}')



2024-05-30 17:06:57.945478: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-30 17:06:57.945655: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-30 17:06:57.945703: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-30 17:06:58.155622: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-30 17:06:58.156032: I tensorflow/compile

Len dataset: 7052
Len all train: 105780


In [12]:
class CustomCosineDecayWithWarmup(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate, decay_steps, alpha=0.0, warmup_learning_rate=0.0, warmup_steps=0, name=None):
        super().__init__()
        self.initial_learning_rate = initial_learning_rate
        self.decay_steps = decay_steps
        self.alpha = alpha
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.name = name

    def __call__(self, step):
        with tf.name_scope(self.name or "CustomCosineDecayWithWarmup"):
            # Lämpenemisvaihe
            learning_rate = tf.cond(
                step < self.warmup_steps,
                lambda: self.warmup_learning_rate + step / self.warmup_steps * (self.initial_learning_rate - self.warmup_learning_rate),
                lambda: self.initial_learning_rate
            )
            # Kosinilasku lämpenemisen jälkeen
            cosine_decay = tf.keras.optimizers.schedules.CosineDecay(
                initial_learning_rate=self.initial_learning_rate,
                decay_steps=self.decay_steps,
                alpha=self.alpha
            )
            decayed_learning_rate = cosine_decay(step - self.warmup_steps)
            
            return tf.cond(step < self.warmup_steps, lambda: learning_rate, lambda: decayed_learning_rate)

    def get_config(self):
        return {
            "initial_learning_rate": self.initial_learning_rate,
            "decay_steps": self.decay_steps,
            "alpha": self.alpha,
            "warmup_learning_rate": self.warmup_learning_rate,
            "warmup_steps": self.warmup_steps,
            "name": self.name
        }

In [ ]:
tf.keras.backend.clear_session()
gc.collect()


def r2_score_tf(y_true, y_pred):

    try: 
        ss_res = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
        ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)
        r2 = 1 - ss_res/(ss_tot + tf.keras.backend.epsilon())
        r2 = tf.where(tf.math.is_nan(r2), tf.zeros_like(r2), r2) 
        return tf.reduce_mean(tf.maximum(r2, 0.0))
    except Exception as e:
        # print(f'Error in r2_score_tf: {e}')
        return float('-inf')



image_input_avg = Input(shape=(480, 480, 3), name='image_input_avg')

eff_avg_base =  ConvNeXtLarge(weights='imagenet', include_top=False, pooling='avg', input_tensor=image_input_avg)

eff_avg_base.trainable = True
for layer in eff_avg_base.layers[:-70]:
    layer.trainable = False

pass_through_layer = Lambda(lambda x: x, name='nas_features')(eff_avg_base.output)

pass_through_layer = Dropout(0.5)(pass_through_layer)

nas_output = Dense(6, activation='linear', name='final_tune_output')(pass_through_layer)

finetune_model = Model(inputs=image_input_avg, outputs=nas_output, name='finetune_model')

lr_schedule = CustomCosineDecayWithWarmup(
    initial_learning_rate=1e-4,
    decay_steps=len_dataset * (EPOCHS - 1),
    alpha=0.0,
    warmup_learning_rate=1e-6,
    warmup_steps=len_dataset,
    name="CosineDecayWithWarmup"
)

finetune_model.compile(optimizer=optimizers.Adam(learning_rate=lr_schedule), loss='mse', metrics=['mse', 'mae', 'mape', r2_score_tf])


In [ ]:
trainable_count = sum([tf.size(v).numpy() for v in finetune_model.trainable_weights])
non_trainable_count = sum([tf.size(v).numpy() for v in finetune_model.non_trainable_weights])
print(f"Total parameters: {trainable_count + non_trainable_count:,}")
print(f"Trainable parameters: {trainable_count:,}")
print(f"Non-trainable parameters: {non_trainable_count:,}")


In [13]:
import tensorflow as tf
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
import io

class TrainImageLoggingCallback(Callback):
    def __init__(self, log_dir, data):
        super(TrainImageLoggingCallback, self).__init__()
        self.log_dir = log_dir
        self.data = data
        self.writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        # Fetch a batch of images
        for imgs, tar in self.data.take(1):  # Adjust depending on your dataset structure
            
            # augmented_images = tf.map_fn(augment_image, imgs)
            augmented_images = tf.cast(imgs, tf.uint8)    
        
            # Prepare the image to write to TensorBoard
            with self.writer.as_default():
                tf.summary.image("Augmented Images", augmented_images, step=epoch, max_outputs=20)

            self.writer.flush()


In [14]:

log_folder = f"./logs/new_runs/trial_{study_name}"

print(f'Logging tensorboard to {log_folder}')
os.makedirs(log_folder, exist_ok=True)

# Aseta logitiedostojen hakemisto
tensorboard_callback = TensorBoard(log_dir=log_folder, histogram_freq=1, update_freq='batch')

Logging tensorboard to ./logs/new_runs/trial_527_convnextl_0


In [ ]:
checkpoint_filepath = './NN_search/model_epoch_{epoch:02d}.h5'

model_checkpoint_callback =  tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    save_freq='epoch',  # Tallenna jokaisen eepokin jälkeen
    verbose=1,  # Näytä viesti tallennuksesta
    save_best_only=False  # Tallenna jokainen eepokki, älä vain parasta
)

callbacks = [
    model_checkpoint_callback,
    tensorboard_callback,
    TrainImageLoggingCallback(log_folder, train_dataset)    
]

history = finetune_model.fit(train_dataset,  epochs=EPOCHS, verbose=1, callbacks=callbacks)




In [15]:
class CustomCosineDecayWithWarmup(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate, decay_steps, alpha=0.0, warmup_learning_rate=0.0, warmup_steps=0, name=None):
        super().__init__()
        self.initial_learning_rate = initial_learning_rate
        self.decay_steps = decay_steps
        self.alpha = alpha
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.name = name

    def __call__(self, step):
        with tf.name_scope(self.name or "CustomCosineDecayWithWarmup"):
            learning_rate = tf.cond(
                step < self.warmup_steps,
                lambda: tf.cast(self.warmup_learning_rate + step / self.warmup_steps * (self.initial_learning_rate - self.warmup_learning_rate), dtype=tf.float32),
                lambda: tf.cast(self.initial_learning_rate, dtype=tf.float32)
            )
            # Kosinilasku lämpenemisen jälkeen
            cosine_decay = tf.keras.optimizers.schedules.CosineDecay(
                initial_learning_rate=tf.cast(self.initial_learning_rate, dtype=tf.float32),
                decay_steps=self.decay_steps,
                alpha=self.alpha
            )
            decayed_learning_rate = cosine_decay(step - self.warmup_steps)
            
            return tf.cond(step < self.warmup_steps, lambda: learning_rate, lambda: decayed_learning_rate)

    def get_config(self):
        return {
            "initial_learning_rate": self.initial_learning_rate,
            "decay_steps": self.decay_steps,
            "alpha": self.alpha,
            "warmup_learning_rate": self.warmup_learning_rate,
            "warmup_steps": self.warmup_steps,
            "name": self.name
        }


In [16]:
lr_schedule_updated = CustomCosineDecayWithWarmup(
    initial_learning_rate=3e-5,
    decay_steps=len_dataset * (7 - 1) ,  # Vähennä jo suoritetut askeleet
    alpha=0.0,
    warmup_learning_rate=1e-5,
    warmup_steps=len_dataset ,  # Vähennä jo suoritetut askeleet, jos warmup_steps > completed_steps
    name="CosineDecayWithWarmup"
)

In [18]:
class LayerScale(layers.Layer):
    """Layer scale module.

    References:
      - https://arxiv.org/abs/2103.17239

    Args:
      init_values (float): Initial value for layer scale. Should be within
        [0, 1].
      projection_dim (int): Projection dimensionality.

    Returns:
      Tensor multiplied to the scale.
    """

    def __init__(self, init_values, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.init_values = init_values
        self.projection_dim = projection_dim

    def build(self, input_shape):
        self.gamma = tf.Variable(
            self.init_values * tf.ones((self.projection_dim,))
        )

    def call(self, x):
        return x * self.gamma

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "init_values": self.init_values,
                "projection_dim": self.projection_dim,
            }
        )
        return config

class StochasticDepth(layers.Layer):
    """Stochastic Depth module.

    It performs batch-wise dropping rather than sample-wise. In libraries like
    `timm`, it's similar to `DropPath` layers that drops residual paths
    sample-wise.

    References:
      - https://github.com/rwightman/pytorch-image-models

    Args:
      drop_path_rate (float): Probability of dropping paths. Should be within
        [0, 1].

    Returns:
      Tensor either with the residual path dropped or kept.
    """

    def __init__(self, drop_path_rate, **kwargs):
        super().__init__(**kwargs)
        self.drop_path_rate = drop_path_rate

    def call(self, x, training=None):
        if training:
            keep_prob = 1 - self.drop_path_rate
            shape = (tf.shape(x)[0],) + (1,) * (len(tf.shape(x)) - 1)
            random_tensor = keep_prob + tf.random.uniform(shape, 0, 1)
            random_tensor = tf.floor(random_tensor)
            return (x / keep_prob) * random_tensor
        return x

    def get_config(self):
        config = super().get_config()
        config.update({"drop_path_rate": self.drop_path_rate})
        return config


def r2_score_tf(y_true, y_pred):

        try: 
            ss_res = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
            ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)
            r2 = 1 - ss_res/(ss_tot + tf.keras.backend.epsilon())
            r2 = tf.where(tf.math.is_nan(r2), tf.zeros_like(r2), r2) 
            return tf.reduce_mean(tf.maximum(r2, 0.0))
        except Exception as e:
            # print(f'Error in r2_score_tf: {e}')
            return float('-inf')


    
custom_objects = {"r2_score_tf": r2_score_tf, "LayerScale": LayerScale, "StochasticDepth": StochasticDepth, "CustomCosineDecayWithWarmup": CustomCosineDecayWithWarmup}



tf.keras.backend.clear_session()
gc.collect()

finetune_model = tf.keras.models.load_model(f'./NN_search/model_epoch_11.h5', custom_objects=custom_objects)



In [19]:
with tf.keras.utils.custom_object_scope({'r2_score_tf': r2_score_tf}):
    finetune_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule_updated),  # Update learning rate as needed
        loss='mse',
        metrics=['mse', 'mae', 'mape', r2_score_tf]
    )

In [ ]:
tf.keras.backend.clear_session()
gc.collect()

history = finetune_model.fit(
    train_dataset,
    epochs=7,  # Määritä kuinka monta lisäeepokkia haluat suorittaa
    verbose=1,
    callbacks=callbacks
    
)

In [ ]:

tf.keras.backend.clear_session()
gc.collect()


class LayerScale(layers.Layer):
    """Layer scale module.

    References:
      - https://arxiv.org/abs/2103.17239

    Args:
      init_values (float): Initial value for layer scale. Should be within
        [0, 1].
      projection_dim (int): Projection dimensionality.

    Returns:
      Tensor multiplied to the scale.
    """

    def __init__(self, init_values, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.init_values = init_values
        self.projection_dim = projection_dim

    def build(self, input_shape):
        self.gamma = tf.Variable(
            self.init_values * tf.ones((self.projection_dim,))
        )

    def call(self, x):
        return x * self.gamma

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "init_values": self.init_values,
                "projection_dim": self.projection_dim,
            }
        )
        return config

class StochasticDepth(layers.Layer):
    """Stochastic Depth module.

    It performs batch-wise dropping rather than sample-wise. In libraries like
    `timm`, it's similar to `DropPath` layers that drops residual paths
    sample-wise.

    References:
      - https://github.com/rwightman/pytorch-image-models

    Args:
      drop_path_rate (float): Probability of dropping paths. Should be within
        [0, 1].

    Returns:
      Tensor either with the residual path dropped or kept.
    """

    def __init__(self, drop_path_rate, **kwargs):
        super().__init__(**kwargs)
        self.drop_path_rate = drop_path_rate

    def call(self, x, training=None):
        if training:
            keep_prob = 1 - self.drop_path_rate
            shape = (tf.shape(x)[0],) + (1,) * (len(tf.shape(x)) - 1)
            random_tensor = keep_prob + tf.random.uniform(shape, 0, 1)
            random_tensor = tf.floor(random_tensor)
            return (x / keep_prob) * random_tensor
        return x

    def get_config(self):
        config = super().get_config()
        config.update({"drop_path_rate": self.drop_path_rate})
        return config


def r2_score_tf(y_true, y_pred):

        try: 
            ss_res = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
            ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)
            r2 = 1 - ss_res/(ss_tot + tf.keras.backend.epsilon())
            r2 = tf.where(tf.math.is_nan(r2), tf.zeros_like(r2), r2) 
            return tf.reduce_mean(tf.maximum(r2, 0.0))
        except Exception as e:
            # print(f'Error in r2_score_tf: {e}')
            return float('-inf')


    
custom_objects = {"r2_score_tf": r2_score_tf, "LayerScale": LayerScale, "StochasticDepth": StochasticDepth, "CustomCosineDecayWithWarmup": CustomCosineDecayWithWarmup}

for epoch in range(1, 12):
    
    tf.keras.backend.clear_session()
    gc.collect()

    model = tf.keras.models.load_model(f'./NN_search/model_epoch_{epoch:02d}.h5', custom_objects=custom_objects)
    preds_transformed = model.predict(valid_dataset, verbose=1)

    for i, target in enumerate(mean_columns):
            scaler = scaler_transforms[target]
            if scaler is not None:
                preds_transformed[:, i] = scaler.inverse_transform(preds_transformed[:, i].reshape(-1, 1)).flatten()     


    for i, target in enumerate(mean_columns):
            method, param = log_transforms[target]
            if method == 'log':
                # Käänteinen logaritmimuunnos
                preds_transformed[:, i] = np.power(param, preds_transformed[:, i])
            elif method == 'power':
                # Käänteinen potenssimuunnos
                preds_transformed[:, i] = np.power(preds_transformed[:, i], 1 / param)


    R2_valid = r2_score(y_valid, preds_transformed)
    MSE_valid = mean_squared_error(y_valid, preds_transformed)
    MAE_valid = mean_absolute_error(y_valid, preds_transformed)
    MAPE_valid = mean_absolute_percentage_error(y_valid, preds_transformed)
    print(f'Study name {study_name}')
    print(f'Model file at epoch {epoch}: ./NN_search/model_epoch_{epoch:02d}_5.h5')
    print(f'Model at epoch {epoch}:\nR2 : {R2_valid:.5f}, MSE : {MSE_valid:.5f}, MAE : {MAE_valid:.5f}, MAPE : {MAPE_valid:.5f}')

    del model 



In [ ]:
finetune_model = tf.keras.models.load_model('./NN_search/model_epoch_11_5.h5', custom_objects=custom_objects)

In [ ]:
# TEST DATA 

tf.keras.backend.clear_session()
gc.collect()

test_df_copy = test_df.copy()
submission_df = test_df_copy[['id']].copy()

test_images_path = test_df_copy['image_path'].values

dummy_y = np.zeros((len(test_df_copy), 6))

test_dataset = tf.data.Dataset.from_tensor_slices((test_images_path, dummy_y))
test_dataset = test_dataset.map(process_path_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(16).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

predictions = finetune_model.predict(test_dataset, verbose=1)

transformation_options = {
    'none': lambda x: x,
    'log': lambda x, base: np.log(x) / np.log(base),
    'power': lambda x, exp: np.power(x, exp)
}


for i, target in enumerate(mean_columns):
        scaler = scaler_transforms[target]
        if scaler is not None:
            predictions[:, i] = scaler.inverse_transform(predictions[:, i].reshape(-1, 1)).flatten()     


for i, target in enumerate(mean_columns):
        method, param = log_transforms[target]
        if method == 'log':
            # Käänteinen logaritmimuunnos
            predictions[:, i] = np.power(param, predictions[:, i])
        elif method == 'power':
            # Käänteinen potenssimuunnos
            predictions[:, i] = np.power(predictions[:, i], 1 / param)


In [ ]:
train_df[mean_columns].describe()

In [ ]:
predictions_df = pd.DataFrame(predictions, columns = mean_columns)
predictions_df.describe()

In [ ]:
print(predictions.shape)

In [ ]:

target_columns = ['X4', 'X11', 'X18', 'X50', 'X26', 'X3112']

submission_df[target_columns] = predictions


In [ ]:
original_data_desc = train_original[mean_columns].describe().to_json()
results_training.at[study_name_result, 'Original data Desc'] = original_data_desc



In [ ]:
print(f'{str(log_transforms.items())}')
print(f'{str(scaler_transforms)}')

results_training.at[study_name_result, 'Scalers'] = f'{scaler_transforms}'
results_training.at[study_name_result, 'Log/Pot transforms'] = f'{str(log_transforms.items())}'



In [ ]:
# results_training.at[study_name_result, 'Kaggle R2'] = None


# # results_training.drop('423_std_powerlog_3', inplace=True)
# # results_training.head()

# for index, row in results_training.iterrows():
#     print(f"Study Name: {index}")
#     print(f'Kaggle R2: {row["Kaggle R2"]}')
#     print(f"Train R2: {row['Train R2']}, Train MSE: {row['Train MSE']}, Train MAE : {row['Train MAE']}, Train MAPE: {row['Train MAPE']}")
#     print(f'Valid R2: {row["Valid R2"]}, Valid MSE: {row["Valid MSE"]}, Valid MAE: {row["Valid MAE"]}, Valid MAPE: {row["Valid MAPE"]}')
#     print("-" * 50)
#     print("Train preds Description:")
#     display(pd.read_json(row['Train preds Desc']))
#     print("Valid preds Description:")
#     display(pd.read_json(row['Valid preds Desc']))
#     print("Test preds Description:")
#     display(pd.read_json(row['Test preds Desc']))
#     print("Original data Description:")
#     display(pd.read_json(row['Original data Desc']))
    

In [ ]:
submission_df.head()

In [ ]:
print(submission_df.info())

submission_df.to_csv('./data/submission.csv', index=False)

In [ ]:
with open(results_training_name, 'wb') as f:
    results_training.to_pickle(f)

In [20]:
feature_model_output = finetune_model.get_layer('nas_features')
feature_model = Model(inputs=finetune_model.input, outputs=feature_model_output.output)


tf.keras.backend.clear_session()
gc.collect()


1853

In [21]:
train_all_paths = train_original['image_path'].values
dummy_y = np.zeros((len(train_original), 6))

train_all_dataset = tf.data.Dataset.from_tensor_slices((train_all_paths, dummy_y))
train_all_dataset = train_all_dataset.map(process_path_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_all_dataset = train_all_dataset.batch(6).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

train_features = feature_model.predict(train_all_dataset, verbose=1)

train_original[f'model_features_{study_name}'] = train_features.tolist()

2024-05-30 17:08:26.511476: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [52885,6]
	 [[{{node Placeholder/_1}}]]
2024-05-30 17:08:26.511698: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [52885,6]
	 [[{{node Placeholder/_1}}]]
2024-05-30 17:08:28.829989: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2024-05-30 17:08:29.558967: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0xbab55fc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:

8503/8815 [===========================>..] - ETA: 1:48

In [ ]:
train_original.head()

In [ ]:
pickle_file_path = f'./data/train_df_neljas.pickle'

print(f'Saving train_df to {pickle_file_path}')
with open(pickle_file_path, 'wb') as f:
    pickle.dump(train_original, f)

In [ ]:

tf.keras.backend.clear_session()
gc.collect()

In [ ]:
test_all_paths = test_df['image_path'].values
dummy_y = np.zeros((len(test_df), 6))

test_all_dataset = tf.data.Dataset.from_tensor_slices((test_all_paths, dummy_y))
test_all_dataset = test_all_dataset.map(process_path_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_all_dataset = test_all_dataset.batch(8).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

test_features = feature_model.predict(test_all_dataset, verbose=1)

test_df[f'model_features_{study_name}'] = test_features.tolist()

In [ ]:
test_df[f'model_features_{study_name}'].head()

In [ ]:
testa = test_df[f'model_features_{study_name}'].iloc[0]

In [ ]:
testa = np.array(testa)
testa.shape

In [ ]:
test_df.head()

In [ ]:
pickle_file_path = f'./data/test_df_neljas.pickle'

print(f'Saving test_df to {pickle_file_path}')
with open(pickle_file_path, 'wb') as f:
    pickle.dump(test_df, f)
